In [1]:
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline
%config lnlineBackend.figure_format = 'retina'

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import keras

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.optimizers import RMSprop, Adam
from keras.layers import Conv2D, MaxPooling2D, Flatten   # required for CNN
from keras import backend as K

from keras import regularizers


Using TensorFlow backend.


In [2]:
# load the raw data
X_train = np.load('X_train.npy')
y_train = np.load('y_train.npy')
print('shape of the training data:', X_train.shape)
print('shape of the target data:', y_train.shape)

shape of the training data: (3313, 150, 150)
shape of the target data: (3313,)


In [3]:
X_train.max()

1

In [4]:
X_train.min()

0

In [5]:
np.unique(X_train)# there are only two values in the X_train

array([0, 1], dtype=int8)

In [6]:
#X_train = X_train.reshape(3313, 150*150)
#y_train=y_train.reshape(3313,1)

In [7]:
3313//5*4

2648

In [8]:
img_rows, img_cols=150,150
x_train, lable_train=X_train[:2648], y_train[:2648]
lable_train=lable_train-1
X_test, y_test=X_train[2648:], y_train[2648:]
y_test-=1
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
lable_train = keras.utils.to_categorical(lable_train, 4)
y_test = keras.utils.to_categorical(y_test, 4)

In [9]:
x_train.shape

(2648, 150, 150, 1)

NN

In [14]:
model = Sequential()
model.add(Conv2D(128,# number of kernels 
                 kernel_size=(3, 3),# kernel size
                 activation='relu',
                 input_shape=(150,150,1)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))#0.25
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

model.summary()

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=Adam(lr=0.002),
              metrics=['accuracy'])

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 148, 148, 128)     1280      
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 146, 146, 32)      36896     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 71, 71, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 35, 35, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 78400)            

In [16]:
batch_size = 128
epochs = 30

from keras.callbacks import EarlyStopping
early_stopping_monitor = EarlyStopping(monitor='val_loss', patience=4)

history = model.fit(x_train, lable_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          callbacks=[early_stopping_monitor],  
          validation_data=(X_test, y_test))


Train on 2648 samples, validate on 665 samples
Epoch 1/30
1664/2648 [=================>............] - ETA: 1:15 - loss: 1.4248 - accuracy: 0.2620

KeyboardInterrupt: 

In [18]:
score = model.evaluate(x_train, lable_train, verbose=1)
print('Test loss: {:.4}'.format(score[0]))
print('Test accuracy: {:.4}'.format(score[1]))

2648/2648 [==============================] - 21s 8ms/step
Test loss: 1.465
Test accuracy: 0.2319


In [83]:
X_t = np.load('X_test.npy')
X_t = X_t.reshape(X_t.shape[0], img_rows, img_cols, 1)
result=model.predict(X_t)
result[0]

array([0.50654733, 0.22549427, 0.26149127, 0.00646712], dtype=float32)

In [84]:
result.shape

(829, 4)

In [85]:
y=[]
for i in result:
    for j in (np.where(i==np.max(i))):
        for m in j:
            y.append(m+1)
resultfinal=np.array(y)

In [86]:
import pandas as pd
train=pd.read_csv('./sample_submission.csv')
train.head()

Id  Category
0   0         4
1   1         2
2   2         2
3   3         1
4   4         3

In [87]:
pid=pd.DataFrame(columns=['Id'],data=np.arange(0,829,1))
category=pd.DataFrame(columns=['Category'],data=resultfinal)
testresult=pid.join(category)

In [88]:
testresult

Id  Category
0      0         1
1      1         3
2      2         4
3      3         4
4      4         1
..   ...       ...
824  824         1
825  825         1
826  826         1
827  827         3
828  828         4

[829 rows x 2 columns]

In [89]:
t=testresult[testresult==train]#1:205 2:198 3:206 4:200 5:229 6:205

In [90]:
t.describe()

Id    Category
count  829.000000  206.000000
mean   414.000000    2.975728
std    239.455981    1.074944
min      0.000000    1.000000
25%    207.000000    2.000000
50%    414.000000    3.000000
75%    621.000000    4.000000
max    828.000000    4.000000

In [124]:
testresult.to_csv('./result.csv',index=False)